In [13]:
import pandas as pd

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import GlobalMaxPooling1D, Dense, Activation, Dropout, Embedding,Conv1D
import random 

In [14]:
def load_json_file(filename):
    with open(filename) as f:
        file = json.load(f)
    return file

filename =r"C:\Users\Rohan\Pictures\rohan\langchine\hh.json"

intents = load_json_file(filename) 

df = pd.DataFrame(columns=['Pattern', 'Tag'])
df.head()

,Pattern,Tag


In [15]:
def extract_json_info(json_file, df):
    # Iterate over each intent in the JSON file
    for intent in json_file['intents']:
        # Iterate over each pattern in the current intent
        for pattern in intent['patterns']:
            # Create a list containing the pattern and its associated tag
            sentence_tag = [pattern, intent['tag']]
            # Append the pattern and tag to the DataFrame
            df.loc[len(df.index)] = sentence_tag
     # Return the updated DataFrame            
    return df

df = extract_json_info(intents, df)
df.head(3)

,Pattern,Tag
0,Hi,greeting
1,How are you?,greeting
2,Is anyone there?,greeting


In [16]:
labels = df['Tag'].unique().tolist()
labels = [s.strip() for s in labels]
labels[:10]

['greeting',
 'goodbye',
 'creator',
 'name',
 'hours',
 'number',
 'course',
 'fees',
 'location',
 'hostel']

In [17]:
df.shape

(405, 2)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 405 entries, 0 to 404
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Pattern  405 non-null    object
 1   Tag      405 non-null    object
dtypes: object(2)
memory usage: 9.5+ KB


In [31]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Pattern'])

# Convert text patterns to numerical sequences
X = tokenizer.texts_to_sequences(df['Pattern'])

# Pad sequences to ensure uniform length
max_sequence_length = max(len(seq) for seq in X)
X_padded = pad_sequences(X, maxlen=max_sequence_length, padding='post')

# Convert tags to numerical labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Tag'])
X_padded

array([[161,   0,   0, ...,   0,   0,   0],
       [ 12,   9,   5, ...,   0,   0,   0],
       [  1, 162,  24, ...,   0,   0,   0],
       ...,
       [ 17,   0,   0, ...,   0,   0,   0],
       [ 17,  13,   0, ...,   0,   0,   0],
       [ 19,   1,   4, ...,   0,   0,   0]])

In [32]:
max_sequence_length 

12

In [33]:
print(X_padded.shape)
print(y.shape)

(405, 12)
(405,)


In [34]:
X_train, X_val, y_train, y_val = train_test_split(X_padded, y, test_size=0.2, random_state=42)
# Define the CNN model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32)

Epoch 1/100
11/11 [==============================] - 2s 52ms/step - loss: 3.6236 - accuracy: 0.0370 - val_loss: 3.5922 - val_accuracy: 0.0617
Epoch 2/100
11/11 [==============================] - 0s 19ms/step - loss: 3.5686 - accuracy: 0.0617 - val_loss: 3.5542 - val_accuracy: 0.0617
Epoch 3/100
11/11 [==============================] - 0s 16ms/step - loss: 3.5428 - accuracy: 0.0741 - val_loss: 3.5217 - val_accuracy: 0.1111
Epoch 4/100
11/11 [==============================] - 0s 22ms/step - loss: 3.5065 - accuracy: 0.1080 - val_loss: 3.4964 - val_accuracy: 0.0864
Epoch 5/100
11/11 [==============================] - 0s 21ms/step - loss: 3.4556 - accuracy: 0.1080 - val_loss: 3.4581 - val_accuracy: 0.0864
Epoch 6/100
11/11 [==============================] - 0s 27ms/step - loss: 3.3765 - accuracy: 0.1296 - val_loss: 3.4159 - val_accuracy: 0.1481
Epoch 7/100
11/11 [==============================] - 0s 23ms/step - loss: 3.2741 - accuracy: 0.1667 - val_loss: 3.3479 - val_accuracy: 0.1481
Epoch 

11/11 [==============================] - 0s 26ms/step - loss: 0.1130 - accuracy: 0.9784 - val_loss: 1.1183 - val_accuracy: 0.7531
Epoch 59/100
11/11 [==============================] - 0s 15ms/step - loss: 0.1498 - accuracy: 0.9660 - val_loss: 1.1119 - val_accuracy: 0.7407
Epoch 60/100
11/11 [==============================] - 0s 27ms/step - loss: 0.0972 - accuracy: 0.9784 - val_loss: 1.1576 - val_accuracy: 0.7284
Epoch 61/100
11/11 [==============================] - 0s 31ms/step - loss: 0.1011 - accuracy: 0.9753 - val_loss: 1.1043 - val_accuracy: 0.7407
Epoch 62/100
11/11 [==============================] - 0s 30ms/step - loss: 0.0959 - accuracy: 0.9846 - val_loss: 1.1143 - val_accuracy: 0.7284
Epoch 63/100
11/11 [==============================] - 0s 27ms/step - loss: 0.1001 - accuracy: 0.9846 - val_loss: 1.1230 - val_accuracy: 0.7160
Epoch 64/100
11/11 [==============================] - 0s 31ms/step - loss: 0.0820 - accuracy: 0.9846 - val_loss: 1.0860 - val_accuracy: 0.7037
Epoch 65/100

In [11]:
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {val_accuracy:.4f}')

6/6 [==============================] - 0s 5ms/step - loss: 0.2781 - accuracy: 0.9444
Validation Accuracy: 0.9444


In [12]:
model.save('chatbot.h5')

In [12]:
from keras.models import load_model
loaded_model = load_model('chatbot.h5')
def preprocess_input_sentence(sentence, tokenizer, max_sequence_length):
    # Tokenize input sentence
    input_sequence = tokenizer.texts_to_sequences([sentence])
    # Pad sequences
    padded_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')
    return padded_sequence

# Function to get response
def get_response(sentence, loaded_model, tokenizer, max_sequence_length, intents):
    # Preprocess input sentence
    preprocessed_input = preprocess_input_sentence(sentence, tokenizer, max_sequence_length)
    # Predict label
    predicted_label = loaded_model.predict(preprocessed_input).argmax(axis=-1)
    # Convert label to tag
    predicted_tag = label_encoder.inverse_transform(predicted_label)
    
    # Iterate through the intents to find the matching tag
    for intent in intents["intents"]:
        if intent["tag"] == predicted_tag[0]:
            # Select a random response from the list of responses
            response = random.choice(intent["responses"])
            return response
    
    # If no matching intent is found, return a default message
    return "I'm sorry, I didn't understand that."


print("Chatbot: Hello! How can I assist you today? (Type 'exit' to end)")
while True:
    # Get user input
    sentence = input("You: ")
    # Check if the user wants to exit
    if sentence.lower() == 'exit':
        print("Chatbot: Goodbye! Have a great day!")
        break
    # Get response from chatbot
    response = get_response(sentence, loaded_model, tokenizer, max_sequence_length, intents)
    print("Chatbot:", response)

Chatbot: Hello! How can I assist you today? (Type 'exit' to end)
You: hii
1/1 [==============================] - 0s 217ms/step
Chatbot: Hi there, how can I help?
You: how to helps me
1/1 [==============================] - 0s 44ms/step
Chatbot: Sad to see you go :(
You: which is best products
1/1 [==============================] - 0s 46ms/step
Chatbot: To know about placement visit <a target="_blank" href="PLACEMENT INFORMATION LINK FROM YOUR UNIVERSITY WEBSITE IF THEY HAVE">here</a>


KeyboardInterrupt: Interrupted by user